In [25]:
TEAMCODE = 'bkft'
TASK = '_igt'
MODEL = 'Trans'

FROMDIR = r'./bkft/'
TODIR = FROMDIR + '/data/'
ANNOTATED_FILE = FROMDIR+'bkft.annotated'
UNANNOTATED_FILE = FROMDIR+'bkft.unannotated'

KEY = TEAMCODE+TASK+MODEL

PUNCTUATION = ['!', '.' ,'“', '”', '"', '…', ')', '(']

In [26]:
def aligned(mrph, gls):
    if not len(mrph.split()) == len(gls.split()):
        return False
    if not len(mrph.split('-')) == len(gls.split('-')):
        return False
    if not len(mrph.split('=')) == len(gls.split('=')):
        return False
    return True

def preprocess(igtlines, annotated=True):
    newlines = []
    unannotated = []
    for line in igtlines:
        if line != '':
            for punct in PUNCTUATION:
                line = line.translate(str.maketrans('', '', punct))
            #print(line) 
            igt = line.split('\n')
            t,m,g = igt[0], igt[1], igt[2]
            notransl = igt[0]+'\n'+igt[1]+'\n'+igt[2]
            if annotated == True and aligned(m,g):
                newlines.append(notransl)
            else:
                unannotated.append(notransl+'\n')
    if annotated:
        return newlines, unannotated
    return unannotated

In [27]:
annotated = open(ANNOTATED_FILE, encoding='utf8').read().split('\n\n')
unannotated_lines = [line.strip() for line in open(UNANNOTATED_FILE, encoding='utf8')]
unannotated = []
for i,line in enumerate(unannotated_lines):
    if line == '':
        unannotated.append(unannotated_lines[i+1])
unannotated = [line+'\n@@@\n@@@\n' for line in unannotated]


annotated, misaligned = preprocess(annotated)
unannotated = preprocess(unannotated, annotated=False) + misaligned

with open(TODIR+KEY+'.predict', 'w', encoding='utf8') as u:
    u.write('\n'.join(unannotated))

test_size = len(annotated)//10
test = annotated[:test_size]
train = annotated[test_size:]

with open(TODIR+KEY+'0'+'.train', 'w', encoding='utf8') as t:
    t.write('\n\n'.join(train))
with open(TODIR+KEY+'0'+'.test', 'w', encoding='utf8') as t:
    t.write('\n\n'.join(test))